In [1]:
from queryportal.subgraphinterface import SubgraphInterface
from datetime import datetime, timedelta

import polars as pl
pl.Config.set_fmt_str_lengths(200)

# HOSTED
sgi = SubgraphInterface(endpoints=['https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum'])

# Decentralized
# sgi = SubgraphInterface(endpoints={
#     # https://thegraph.com/explorer/subgraphs/ELUcwgpm14LKPLrBRuVvPvNKHQ9HvwmtKgKSH6123cr7?view=Overview&chain=mainnet
#     'uniswap-v3-ethereum': 'https://api.playgrounds.network/v1/proxy/subgraphs/id/ELUcwgpm14LKPLrBRuVvPvNKHQ9HvwmtKgKSH6123cr7'
#     })

In [4]:
# print subgraph keys
univ3_dict = sgi.subject.getQueryPaths(sgi.subject.subgraphs['uniswap-v3-ethereum'], 'swaps')


# print fields for swaps entity
print(f'my dict fields for univ3_decentralized swaps entity: {list(univ3_dict.keys())}')

# there appears to be something corrupted in these fieldpaths...
# query_paths = ['hash', 'to', 'from', 'blockNumber', 'timestamp', 'tokenIn', 'amountIn', 'amountInUSD', 'tokenOut', 'amountOut', 'amountOutUSD', 'pool']

query_paths = [
    'hash',
    # 'to',
    # 'from',
    'blockNumber',
    'pool_inputTokenBalances',
    'timestamp',
    'tokenIn_symbol',
    'tokenOut_symbol',
    'amountIn',
    'amountOut',
    'pool_id'
]

filter = {
    'timestamp_gte': int((datetime(2023, 5, 17).timestamp())),
    'timestamp_lte': int(datetime(2023, 5, 18).timestamp()),
    # 'amountInUSD_gte': 10,
    # 'amountOutUSD_gte': 10
    # 'hash': "0xb2d071e74709bddc8ab005aa42ce4251ad77453fed195f856b46d055c88cb556"
    "pool_id": "0x5777d92f208679db4b9778590fa3cab3ac9e2168",
}

query_size = 2250

univ3 = sgi.query_entity(
    query_size=query_size,
    entity='swaps',
    name='uniswap-v3-ethereum', 
    query_paths=query_paths,
    filter_dict = filter,
    orderBy='timestamp',
    block_filter = {'number_gte': 17315061},
    graphql_query_fmt=True
    # saved_file_name='univ3_swaps'
    )


print(univ3.shape[0])


my dict fields for univ3_decentralized swaps entity: ['id', 'hash', 'nonce', 'logIndex', 'gasLimit', 'gasUsed', 'gasPrice', 'protocol', 'account', 'pool', 'blockNumber', 'timestamp', 'tick', 'tokenIn', 'amountIn', 'amountInUSD', 'tokenOut', 'amountOut', 'amountOutUSD']
Querying endpoint: uniswap-v3-ethereum
debug block filter: {'number_gte': 17315061}
graphql query path: query {
  x0e9dac261fb8eb79: swaps(first: 2250, orderBy: timestamp, where: {timestamp_gte: "1684296000", timestamp_lte: "1684382400", pool_: {id: "0x5777d92f208679db4b9778590fa3cab3ac9e2168"}}, block: {number_gte: 17315061}) {
    hash
    blockNumber
    timestamp
    tokenIn {
      symbol
    }
    tokenOut {
      symbol
    }
    amountIn
    amountOut
    pool {
      inputTokenBalances
      id
    }
  }
}
Shape: (477, 10)
Schema: {'hash': Utf8, 'blockNumber': Int64, 'amountIn': Float64, 'amountOut': Float64, 'timestamp': Int64, 'tokenIn_symbol': Utf8, 'tokenOut_symbol': Utf8, 'pool_id': Utf8, 'pool_inputTokenBa

In [5]:
univ3

hash,blockNumber,amountIn,amountOut,timestamp,tokenIn_symbol,tokenOut_symbol,pool_id,pool_inputTokenBalances_0,pool_inputTokenBalances_1
str,i64,f64,f64,i64,str,str,str,f64,f64
"""0x77b1c3ff7c24b7e8dd1bcdad0a8b7799c035de5b996dbf296810e7c27cdf054f""",17276872,5.0000e21,4.9995e9,1684296263,"""DAI""","""USDC""","""0x5777d92f208679db4b9778590fa3cab3ac9e2168""",5.1918e25,5.5890e13
"""0x7e22c38a535a8b62b0340f6478008671263687b1bf0b5c5977e2f65e6fbb5bb6""",17276881,3.0706e9,3.0703e21,1684296371,"""USDC""","""DAI""","""0x5777d92f208679db4b9778590fa3cab3ac9e2168""",5.1918e25,5.5890e13
"""0x2f731158ee274203a208bde5c026655c7c4f49b791b09416fbbf8e239e587cf9""",17276938,2.1642e9,2.1639e21,1684297067,"""USDC""","""DAI""","""0x5777d92f208679db4b9778590fa3cab3ac9e2168""",5.1918e25,5.5890e13
"""0xcae3a6b81440315b1f73c2b4a20d20cef86fff123d91f5e8af2ff1b531fd6f7a""",17276951,4.1770e9,4.1766e21,1684297223,"""USDC""","""DAI""","""0x5777d92f208679db4b9778590fa3cab3ac9e2168""",5.1918e25,5.5890e13
"""0x5d2481c3321bee56207ea1db0842a8d88ead99678753e4633e98c74a97cb6c0f""",17276984,2e8,1.9998e20,1684297619,"""USDC""","""DAI""","""0x5777d92f208679db4b9778590fa3cab3ac9e2168""",5.1918e25,5.5890e13
"""0x3ee86330dff24348e2310e3ca86ab070b30f3af0890722af164e9715aa189fc9""",17276992,5.0000e21,4.9995e9,1684297715,"""DAI""","""USDC""","""0x5777d92f208679db4b9778590fa3cab3ac9e2168""",5.1918e25,5.5890e13
"""0xfa6edc1528b8c9eff5339273f85aa909b5afe02e3dac0637bbb8ad6a19718565""",17276992,1.5002e11,1.5000e23,1684297715,"""USDC""","""DAI""","""0x5777d92f208679db4b9778590fa3cab3ac9e2168""",5.1918e25,5.5890e13
"""0x25ecda42679405c225652d49624093d8f94a5cfe796ebcec4974fae04df26528""",17277000,3.6436e9,3.6432e21,1684297811,"""USDC""","""DAI""","""0x5777d92f208679db4b9778590fa3cab3ac9e2168""",5.1918e25,5.5890e13
"""0xba3c9bee19e7f9493b87f776e0c54a552002412a0bede43b54012f381ba5b0bc""",17277004,2.95098444e8,2.9507e20,1684297859,"""USDC""","""DAI""","""0x5777d92f208679db4b9778590fa3cab3ac9e2168""",5.1918e25,5.5890e13
